In [ ]:
!pip install pulp
!pip install matplotlib

In [54]:
import numpy as np
import random
from pulp import *

In [17]:
#Function for normal distribution truncation:
from scipy.stats import truncnorm

def get_truncated_normal(mean, sd, low, upp):
    return truncnorm((low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

In [30]:
#Function to get the one-hot-encoded vectors for departure and arrival airports:

def one_hot_encode_airport(airport, num_airports):
    encoding = np.zeros(num_airports)
    encoding[airport] = 1
    return encoding

In [29]:
#Generate full info for the arrival sides:

def generate_info_arv(requests):
    ts_arv = np.empty(shape=(len(requests),), dtype='object')
    for i in range(len(requests)):
        ts_arv[i] = requests[i][1] + requests[i][4]/5
        if ts_arv[i] > 287:
            ts_arv[i] = ts_arv[i] - 287
    return ts_arv

In [47]:
#Modify the distribution based on historical data later:
def generate_scenario(number_of_requests, num_airports):

    #number_of_requests = 15000
    ts_72 = get_truncated_normal(mean=72, sd=12, low=0, upp=287).rvs(int(round(number_of_requests/2)))
    ts_72 = np.round(ts_72)

    ts_216 = get_truncated_normal(mean=216, sd=12, low=0, upp=287).rvs(int(round(number_of_requests/2)))
    ts_216 = np.round(ts_216)

    ts_dep = np.concatenate((ts_72, ts_216))
    ts_dep = ts_dep.astype(int)

    #Generate index for requests:

    index = np.array(list(range(number_of_requests)))

    #Generate origin (0 and 1 are two considered origin airports, 2 represent other airports, encoded in one-hot vector):

    #num_airports = 3
    origin_airport = np.empty(shape=(number_of_requests,), dtype='object')
    destination_airport = np.empty(shape=(number_of_requests,), dtype='object')
    for i in range(number_of_requests):
        _org_airport = one_hot_encode_airport(random.randint(0,1), num_airports)
        _org_airport_list = _org_airport.tolist()
        origin_airport[i] = _org_airport_list
        #Generate destination (the destination will be different with the origin):
        _dest_airport = _org_airport.copy()
        while np.array_equal(_dest_airport, _org_airport):
            np.random.shuffle(_dest_airport)
        _dest_airport_list = _dest_airport.tolist()
        destination_airport[i] = _dest_airport_list

    #Generate flying time (assume between airport 0 and 1 is 2 hour, 0 to 2 and 1 to 2 is arbitrary):

    fly_time = np.empty(shape=(number_of_requests,), dtype='object')
    for i in range (number_of_requests):
        if origin_airport[i] == list([1.0, 0.0]) and destination_airport[i] == list([0.0, 1.0]):
            fly_time[i] = 120
        elif origin_airport[i] == list([0.0, 1.0]) and destination_airport[i] == list([1.0, 0.0]):
            fly_time[i] = random.choice([60, 120, 180])

    #Generate status cap:

    status_cap_dep = np.full((number_of_requests,), 0)
    status_cap_arv = np.full((number_of_requests,), 0)


    requests = np.stack((index, ts_dep, origin_airport, destination_airport, fly_time, status_cap_dep), axis=1)

    #Generate full info for the arv side:

    ts_arv = generate_info_arv(requests)

    #pseudo_belong_dep = np.full((number_of_requests,), 0)
    #pseudo_belong_arv = np.full((number_of_requests,), 0)

    # Define requests_full as dtype object
    num_entries = len(index)  # Given that 'index' is defined using np.array(list(range(number_of_requests)))
    # Create an empty array of the desired shape with dtype=object
    requests_full = np.empty((num_entries, 8), dtype=object)
    # Fill the array
    data = [index, ts_dep, origin_airport, destination_airport, fly_time, status_cap_dep, ts_arv, status_cap_arv]
    for i, column_data in enumerate(data):
        requests_full[:, i] = column_data

    return requests_full

In [46]:
def generate_scenario_MILP(requests_full): #0 - departure slot, 1 - arrival slot, 2 - departure airport, 3 - arrival airport

  updated_requests = []

  for req in requests_full:
    updated_req = []
    updated_req.append(req[1])
    updated_req.append(req[6])
    updated_req.append(req[2].index(1))
    updated_req.append(req[3].index(1))
    updated_requests.append(updated_req)

  return updated_requests

In [ ]:
num_airports = 2
number_of_requests = 1000
requests = generate_scenario(number_of_requests, num_airports)
flight_requests = generate_scenario_MILP(requests)
time_slots = 288 # Time slots and their characteristics
capacity_per_slot = 6
max_movements = 6
arrival_departure = [0,1]

# Create a MILP problem
problem = LpProblem(name="Flight_Scheduling", sense=LpMinimize)

# Creating binary decision variables for each flight request, airport, and arrival/departure slot
x = {(req, flight_type, airport, slot): LpVariable(
        name=f"x_{req}_{airport}_{slot}_{arrival_departure}", cat="Binary")
     for req in range(number_of_requests)
     for flight_type in arrival_departure #0 - departure, 1 - arrival
     for airport in range(num_airports)
     for slot in range(time_slots)}

# Objective function - minimises the total absolute difference between the requested and allocated time interval
problem += lpSum(
    abs(x[req, flight_type, airport, slot] * slot - flight_requests[req][flight_type])
    * x[req, flight_type, airport, slot]
    for req in range(number_of_requests)
    for flight_type in arrival_departure
    for airport in range(num_airports)
    for slot in range(time_slots)
)

# Constraints -
# ensure that only one slot is assigned to a flight and no flight can arrive earlier or depart earlier than the beginning of the day.
for req in range(number_of_requests):
  for flight_type in arrival_departure:
      problem += lpSum(x[req, flight_type, airport, 0] for airport in range(num_airports)) == 1

for req in range(number_of_requests):
  for flight_type in arrival_departure:
    for airport in range(num_airports):
      for slot in range(1, time_slots):
        problem += x[req, flight_type, airport, slot] <= x[req, flight_type, airport, slot - 1]

# ensure that every flight has arrived or departed at the end of the scheduling day
for req in range(number_of_requests):
  for flight_type in arrival_departure:
      problem += lpSum(x[req, flight_type, airport, -1] for airport in range(num_airports)) == 0

# impose interval restriction between arrival and departure flight
for req in range(number_of_requests):
    for dep_airport in range(num_airports):
      for arv_airport in range(dep_airport + 1, num_airports):
        problem += lpSum(abs(sum(x[req, 0, dep_airport, slot] for slot in range(time_slots)) - sum(x[req, 1, arv_airport, slot] for slot in time_slots)) * x[req, arrival_departure[0], dep_airport, 0] * x[req, arrival_departure[1], arv_airport, 0]) == flight_requests[req][0] - flight_requests[req][1]

# airport capacity constraints, which limit the number of arrivals and departures at airports
for airport_slot in time_slots:
  for airport in range(num_airports):
    problem += lpSum(int(sum(x[req, flight_type, airport, slot] for slot in range(time_slots)) == airport_slot) for req in range(number_of_requests) for flight_type in arrival_departure) <= capacity_per_slot

# Solve the MILP problem
problem.solve()

updated_slots = []
max_change = float('-inf')
number_unchanged = 0

# Iterate through the decision variables and check if they are equal to 1
for req in range(number_of_requests):
  for flight_type in arrival_departure:
    change = 0
    for airport in range(num_airports):
        if x[req, flight_type, airport, 0].varValue == 1:
            airport_slot = sum(x[req, flight_type, airport, slot] for slot in range(time_slots))
            change = abs(flight_requests[req][flight_type] - airport_slot)
            if change > max_change:
                max_change = change
            if change:
              break
    if not change:
      number_unchanged += 1
    else:
        updated_slots.append((req, flight_type, airport_slot, change))

# Print the slot changes
for entry in updated_slots:
    print(f"Flight Request {entry[0]}: Change from slot {flight_requests[entry[0]][entry[1]]} to {flight_requests[entry[2]]}" + "departure" if not entry[1] else "arrival")

print("Status:", LpStatus[problem.status])
print(f"{number_unchanged} flight requests out of {number_of_requests} were not shifted.")
print("Maximum Shift:", max_change)
print("Objective Value:", problem.objective.value())